In [1]:
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml
from scipy.optimize import fmin_l_bfgs_b


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
Data, labels = ml.loadFile('./Train.txt')

In [3]:
data_g = ml.gaussianize(Data)
data_z = ml.z_normalization(Data)

In [5]:
def init_GMM(DTR, LTR, DTE = None, n_repetions, prior):
    N = DTR.shape[1]
    t = 1e-6
    w = 1.0
    alpha = 0.1
    final_scores = {}
    gmm = {}
    
    mu, C = ml.MU_Cov_calculator(DTR, LTR)
    
    for i in numpy.unique(LTR):
        gmm[i] = [w, mu[i], C[i]]
        GMM_final = {0 : gmm[i]}
        
        for j in range (n_repetions):
            GMM_init = GMM_final.copy()
            
            for i, g in enumerate(GMM_init.values()):
                w, mu, C = g
                newW = w/2
                U, s, Vh = numpy.linalg.svd(C)
                d = U[:, 0:1]*s[0]**0.5*alpha
                newMu1 = mu+d
                newMu2 = mu-d
                GMM_final[i*2] = [newW, newMu1, C]
                GMM_final[i*2+1] = [newW, newMu2, C]
                
            GMM_final, opt_ll = ml.optimize_GMM(DTE, GMM_final)
        final_scores[i] = opt_ll
    
    return list(final_scores[1]-final_scores[0])

In [99]:
g = init_GMM(Data, labels)
define_GMMs(g, 3)

[[1.0, array([[110.86274796],
       [ 46.46524273],
       [  0.4872975 ],
       [  1.84146135],
       [ 12.66631061],
       [ 26.24642558],
       [  8.33306583],
       [105.41024504]]), array([[ 6.66336339e+02,  9.90419182e+01, -2.46369112e+01,
        -1.22903841e+02, -2.32928346e+02, -1.62054057e+02,
         2.81787813e+01,  4.09822277e+02],
       [ 9.90419182e+01,  4.67249473e+01, -3.96934378e+00,
        -2.38128978e+01, -2.38955918e+00, -8.23669357e+00,
         1.21631590e+00,  2.37588198e+01],
       [-2.46369112e+01, -3.96934378e+00,  1.18519822e+00,
         6.57854948e+00,  1.35677300e+01,  9.50681067e+00,
        -1.71873656e+00, -2.55160558e+01],
       [-1.22903841e+02, -2.38128978e+01,  6.57854948e+00,
         4.05904980e+01,  7.90729282e+01,  5.32686957e+01,
        -9.69564905e+00, -1.42530180e+02],
       [-2.32928346e+02, -2.38955918e+00,  1.35677300e+01,
         7.90729282e+01,  8.75980277e+02,  4.59625448e+02,
        -8.28243152e+01, -1.12386364e+03],
  

TypeError: cannot unpack non-iterable int object